# Forecasting Molecule

In [15]:
pip install --upgrade pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler


## Preprocessing

In [2]:
df = pd.read_excel('../test_data_working_students.xlsx')
df.head()

,Record ID,year,month,MoleculeName,TradeName,ProductName,Packs,Value
0,1,2017,12,molecule1,brand1,SKU1,2966.0,32504.671461
1,2,2017,12,molecule2,brand2,SKU2,7675.0,5627.328111
2,3,2017,12,molecule2,brand2,SKU3,24404.0,10741.583225
3,4,2017,12,molecule3,brand3,SKU4,14775.0,65098.087341
4,5,2017,12,molecule3,brand3,SKU5,18206.0,61614.771981


In [ ]:
df['date'] = pd.to_datetime(df[['year', 'month']].assign(day=1))
# df.set_index('date', inplace=True)

# df_monthly = df.groupby([df.index])['Value'].sum()

df['month_of_year'] = df['month']     
df['quarter'] = df['date'].dt.quarter  
df['week_of_year'] = df['date'].dt.isocalendar().week  


df['Value_Lag1']= df.groupby(['MoleculeName', 'TradeName', 'ProductName'])['Value'].shift(1)
df['Packs_Lag1']= df.groupby(['MoleculeName', 'TradeName', 'ProductName'])['Packs'].shift(1)     
  
df['Value_Rolling_3'] = df.groupby(['MoleculeName', 'TradeName', 'ProductName'])['Value'].transform(lambda x: x.rolling(3).mean()) 
df['Packs_Rolling_3'] = df.groupby(['MoleculeName', 'TradeName', 'ProductName'])['Packs'].transform(lambda x: x.rolling(3).mean())   
df['Value_Rolling_6'] = df.groupby(['MoleculeName', 'TradeName', 'ProductName'])['Value'].transform(lambda x: x.rolling(6).mean()) 
df['Packs_Rolling_6'] = df.groupby(['MoleculeName', 'TradeName', 'ProductName'])['Packs'].transform(lambda x: x.rolling(6).mean())    

#drop the nan values from lags/rolling transformations
df.dropna(subset=['Value_Lag1', 'Packs_Lag1', 'Value_Rolling_3', 'Packs_Rolling_3', 'Value_Rolling_6', 'Packs_Rolling_6'], inplace=True)   

 
encoder= OneHotEncoder(sparse=False)
encoded_features = encoder.fit_transform(df[['MoleculeName', 'TradeName', 'ProductName']])
df = df.reset_index(drop=True)
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(['MoleculeName', 'TradeName', 'ProductName']))
df= pd.concat([df, encoded_df], axis=1)   



c:\Users\annap\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [19]:
target_value = 'Value'
target_packs = 'Packs'  

features = ['year', 'month','month_of_year','quarter', 'week_of_year', 'Value_Lag1','Packs_Lag1','Value_Rolling_3', 'Packs_Rolling_3', 'Value_Rolling_6','Packs_Rolling_6'] + list(encoded_df.columns)   

X= df[features]   
y_value = df[target_value]  
y_packs = df[target_packs]        

tscv = TimeSeriesSplit(n_splits=5) 

scaler = StandardScaler()


## Models

In [20]:
def evaluate_model(model, X, y, tscv, param_grid=None):
    mse_scores = []   
    for train_idx, test_idx in tscv.split(X):
        X_train, X_test= X.iloc[train_idx], X.iloc[test_idx] 
        y_train, y_test= y.iloc[train_idx], y.iloc[test_idx] 

        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        print(f"Training set size: {X_train_scaled.shape},  Test set size: {X_test_scaled.shape}")


        # X_train = X_train.values
        # X_test = X_test.values
        # X.columns = X.columns.astype(str)

        # print(f"Training set size: {X_train.shape},  Test set size: {X_test.shape}")

        if param_grid is not None: 
            grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, scoring='neg_mean_squared_error')
            grid_search.fit(X_train_scaled, y_train)
            best_model = grid_search.best_estimator_
            print(f"Best parameters: {grid_search.best_params_}")
        else:
            best_model=model 
            best_model.fit(X_train_scaled, y_train)
            
        y_pred = best_model.predict(X_test_scaled)
        mse = mean_squared_error(y_test, y_pred) 
        mse_scores.append(mse)  
    
    mean_mse = np.mean(mse_scores)    
    print(f'Mean MSE:  {mean_mse}')  
    return np.mean(mse_scores)     


### XGBoost

In [ ]:
X['week_of_year'] = X['week_of_year'].astype('int32')
X = X.astype({col: 'int32' for col in X.select_dtypes(include='Int64').columns})

# print(X.dtypes)
# print(X.isnull().sum()) 
X.fillna(0, inplace=True)

non_numeric_cols = X.select_dtypes(exclude=['number']).columns
# print("Non-numeric columns:", non_numeric_cols)

param_grid_xgboost = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0]
}



C:\Users\annap\AppData\Local\Temp\ipykernel_7320\2410781237.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['week_of_year'] = X['week_of_year'].astype('int32')


In [ ]:
xg_reg_value = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, enable_categorical=False)
xgboost_mse_value = evaluate_model(xg_reg_value, X, y_value, tscv, param_grid_xgboost)
print(f'XGBoost MSE for Value: {xgboost_mse_value}')

Training set size: (586, 247),  Test set size: (581, 247)
Best parameters: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}
Training set size: (1167, 247),  Test set size: (581, 247)
Best parameters: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 200, 'subsample': 0.8}
Training set size: (1748, 247),  Test set size: (581, 247)
Best parameters: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
Training set size: (2329, 247),  Test set size: (581, 247)
Best parameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100, 'subsample': 0.8}
Training set size: (2910, 247),  Test set size: (581, 247)
Best parameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200, 'subsample': 0.8}
Mean MSE:  5967113781.638832
XGBoost MSE for Value: 5967113781.638832


Training set size: (586, 247),  Test set size: (581, 247)
Training set size: (1167, 247),  Test set size: (581, 247)
Training set size: (1748, 247),  Test set size: (581, 247)
Training set size: (2329, 247),  Test set size: (581, 247)
Training set size: (2910, 247),  Test set size: (581, 247)
Mean MSE:  5523031509.515505
XGBoost MSE for Value: 5523031509.515505      --> since this is so big, lets normalize the features

### Random Forest

In [28]:
param_grid_rf = {
    'n_estimators': [100, 200],
    'max_depth': [None, 3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

In [29]:
rf_reg_packs = RandomForestRegressor(n_estimators=100)
rf_mse_packs = evaluate_model(rf_reg_packs, X, y_packs, tscv, param_grid_rf)
print(f'Random Forest MSE for Packs: {rf_mse_packs}')

Training set size: (586, 247),  Test set size: (581, 247)
Best parameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Training set size: (1167, 247),  Test set size: (581, 247)
Best parameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Training set size: (1748, 247),  Test set size: (581, 247)
Best parameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
Training set size: (2329, 247),  Test set size: (581, 247)
Best parameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Training set size: (2910, 247),  Test set size: (581, 247)
Best parameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
Mean MSE:  141525351.4597588
Random Forest MSE for Packs: 141